In [1]:
!pip install empath
import pickle
import numpy as np
import nltk 
import string 
import re 
import inflect 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import base64

from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from empath import Empath
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble.forest import ExtraTreesClassifier
import time
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
lexicon = Empath()
p = inflect.engine() 

import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/pickles.zip", 'r')
zip_ref.extractall("./")

regE = joblib.load('regressorEmpath.pkl')  
regTS = joblib.load('regressorTS20.pkl')
regHy = joblib.load('regressorHybrid.pkl')
gbcE = joblib.load('gbcEmpath.pkl')  
gbcTS = joblib.load('gbcTS20.pkl')
gbcHy = joblib.load('gbcHybrid.pkl')

# Convert to lowercase
def text_lowercase(text): 
	return text.lower() 
  
# convert number into words 
def convert_number(text): 
    # split string into list of words 
    temp_str = text.split() 
    # initialise empty list 
    new_string = [] 
  
    for word in temp_str: 
        # if word is a digit, convert the digit 
        # to numbers and append into the new_string list 
        if word.isdigit(): 
            temp = p.number_to_words(word) 
            new_string.append(temp) 
  
        # append the word as it is 
        else: 
            new_string.append(word) 
  
    # join the words of new_string to form a string 
    temp_str = ' '.join(new_string) 
    return temp_str 

# remove punctuation 
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

# remove whitespace from text 
def remove_whitespace(text): 
    return  " ".join(text.split()) 
  
# remove stopwords function 
def remove_stopwords(text): 
    stop_words = set(stopwords.words("english")) 
    word_tokens = word_tokenize(text) 
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    return ' '.join(filtered_text)

# stem words in the list of tokenised words 
def stem_words(text): 
    word_tokens = word_tokenize(text) 
    stems = [stemmer.stem(word) for word in word_tokens] 
    return ' '.join(stems)

# lemmatize string 
def lemmatize_word(text): 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    return ' '.join(lemmas)

def empath_feature_calc(text):
    try:
        return lexicon.analyze(text,normalize=True)
    except:
        return None

def empath_feature_calc(text):
    try:
        return lexicon.analyze(text,normalize=True)
    except:
        print("gajab bejjati he")
        return None

def fun(res):
  print(res)
  if res == 0:
    return "Fake"
  return "True"

def predictEmpath(text, algorithm):
  df = pd.DataFrame([[text]],
     index=[0],
     columns=['text'])
  df['text']=df['text'].apply(text_lowercase)
  df['text']=df['text'].apply(convert_number)
  df['text']=df['text'].apply(remove_punctuation)
  df['text']=df['text'].apply(remove_whitespace)
  df['text']=df['text'].apply(remove_stopwords)
  df['text']=df['text'].apply(stem_words)
  df['text']=df['text'].apply(lemmatize_word)

  df['empath_features'] = df['text'].apply(empath_feature_calc)

  newDF = pd.DataFrame(columns = list(df.empath_features[0].keys()))
  exclude_idx = []
  for i in range(df.shape[0]):
      dict_new_dataframe = df.empath_features[i]
      try:
          new_df = pd.DataFrame.from_dict(dict_new_dataframe.items()).transpose()
          header_new_dataframe = new_df.iloc[0]  # 0th index is header
          new_df = new_df[1:]  # fetch data except header
          new_df.columns = header_new_dataframe #set header as dataframe header in pandas
          newDF = newDF.append(new_df, ignore_index=True) # populate new dataframe with new data
      except:
          exclude_idx.append(i)
          pass

  columns=newDF.columns[:]
  x1=newDF[columns]
  print(x1)
  if algorithm == "Regressor":
    return fun(regE.predict(x1)[0])
  return fun(gbcE.predict(x1)[0])

def predictTS20(text, algorithm):
  df = pd.DataFrame([[text]],
     index=[0],
     columns=['text'])
  df['text']=df['text'].apply(text_lowercase)
  df['text']=df['text'].apply(convert_number)
  df['text']=df['text'].apply(remove_punctuation)
  df['text']=df['text'].apply(remove_whitespace)
  df['text']=df['text'].apply(remove_stopwords)
  df['text']=df['text'].apply(stem_words)
  df['text']=df['text'].apply(lemmatize_word)
  vectorizer = TfidfVectorizer(stop_words="english", use_idf=True, ngram_range=(1,1))
  tfidf = vectorizer.fit_transform(df['text'])
  n_topics = 20
  nmf = NMF(n_components=n_topics,random_state=0)
  topics = nmf.fit_transform(tfidf)
  top_n_words = 5
  t_words, word_strengths = {}, {}
  for t_id, t in enumerate(nmf.components_):
      t_words[t_id] = [vectorizer.get_feature_names()[i] for i in t.argsort()[:-top_n_words - 1:-1]]
      word_strengths[t_id] = t[t.argsort()[:-top_n_words - 1:-1]]
  pipe = Pipeline([
    ('tfidf', vectorizer),
    ('nmf', nmf)
  ])
  t = pipe.transform(df['text']) 
  t = pd.DataFrame(t, columns=[str(t_words[i]) for i in range(0, n_topics)])

  if algorithm == "Regressor":
    return fun(regTS.predict(t)[0])
  return fun(gbcTS.predict(t)[0])

def predictHybrid(text, algorithm):
  df = pd.DataFrame([[text]],
     index=[0],
     columns=['text'])
  df['text']=df['text'].apply(text_lowercase)
  df['text']=df['text'].apply(convert_number)
  df['text']=df['text'].apply(remove_punctuation)
  df['text']=df['text'].apply(remove_whitespace)
  df['text']=df['text'].apply(remove_stopwords)
  df['text']=df['text'].apply(stem_words)
  df['text']=df['text'].apply(lemmatize_word)

  df['empath_features'] = df['text'].apply(empath_feature_calc)

  newDF = pd.DataFrame(columns = list(df.empath_features[0].keys()))
  exclude_idx = []
  for i in range(df.shape[0]):
      dict_new_dataframe = df.empath_features[i]
      try:
          new_df = pd.DataFrame.from_dict(dict_new_dataframe.items()).transpose()
          header_new_dataframe = new_df.iloc[0]  # 0th index is header
          new_df = new_df[1:]  # fetch data except header
          new_df.columns = header_new_dataframe #set header as dataframe header in pandas
          newDF = newDF.append(new_df, ignore_index=True) # populate new dataframe with new data
      except:
          exclude_idx.append(i)
          pass
  
  vectorizer = TfidfVectorizer(stop_words="english", use_idf=True, ngram_range=(1,1))
  tfidf = vectorizer.fit_transform(df['text'])
  n_topics = 20
  nmf = NMF(n_components=n_topics,random_state=0)
  topics = nmf.fit_transform(tfidf)
  top_n_words = 5
  t_words, word_strengths = {}, {}
  for t_id, t in enumerate(nmf.components_):
      t_words[t_id] = [vectorizer.get_feature_names()[i] for i in t.argsort()[:-top_n_words - 1:-1]]
      word_strengths[t_id] = t[t.argsort()[:-top_n_words - 1:-1]]
  pipe = Pipeline([
    ('tfidf', vectorizer),
    ('nmf', nmf)
  ])
  t = pipe.transform(df['text']) 
  t = pd.DataFrame(t, columns=[str(t_words[i]) for i in range(0, n_topics)])
  merged=pd.concat([t, newDF], axis=1)

  columns=merged.columns[:]
  x0=merged[columns]
  if algorithm == "Regressor":
    return fun(regHy.predict(x0)[0])
  return fun(gbcHy.predict(x0)[0])

import base64
def stringToBase64(s):
    return base64.b64encode(s.encode('utf-8'))

def base64ToString(b):
    return base64.b64decode(b).decode('utf-8')

     |████████████████████████████████| 61kB 4.3MB/s 
  Created wheel for empath: filename=empath-0.89-cp37-none-any.whl size=57824 sha256=039570af764bb599f6c398b75aeee7748dcfcebe4f4db6550189ee565fab78e7
  Stored in directory: /root/.cache/pip/wheels/84/ea/2f/2bc54d4f9985ce61753ebc5b00cb2df51d855589267c667308
Successfully built empath


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Mounted at /content/drive/


In [2]:
!pip install flask-ngrok
from flask import request,make_response
from flask import Flask
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app) 
import json
@app.route("/")
def predict():
  algo = request.args.get('algo')
  granularity = request.args.get('granularity')
  text = base64ToString(request.args.get('text'))
  print(text)
  print(algo)
  if granularity == "Fine" :
      ans = predictEmpath(text, algo)
  elif granularity == "Coarse":
      ans = predictTS20(text, algo)
  else:
      ans = predictHybrid(text, algo)
  print(ans)
  answer=json.dumps(ans)
  res=make_response(answer)
  res.headers['Access-Control-Allow-Origin'] = '*'
  res.headers['Access-Control-Allow-Headers']= "Origin, X-Requested-With, Content-Type, Accept"
  return res
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bb1dc8950892.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Circulating on social networks a video that shows an excerpt from a Spanish television show, supposedly issued December 24, 2019, in which it appears a
woman (who claims to be psychic) to make "predictions". In this video, the woman describes a set of events that have been interpreted as a detailed forecast of
Covid-19 pandemic that has hit the world. It is, however, a fake video, at least as regards the date of issue. The video has been being disseminated on the
Internet with a date and not tam
pered with the real.
Regressor


127.0.0.1 - - [13/May/2021 14:10:03] "GET /?algo=Regressor&granularity=Hybrid&text=Q2lyY3VsYXRpbmcgb24gc29jaWFsIG5ldHdvcmtzIGEgdmlkZW8gdGhhdCBzaG93cyBhbiBleGNlcnB0IGZyb20gYSBTcGFuaXNoIHRlbGV2aXNpb24gc2hvdywgc3VwcG9zZWRseSBpc3N1ZWQgRGVjZW1iZXIgMjQsIDIwMTksIGluIHdoaWNoIGl0IGFwcGVhcnMgYQp3b21hbiAod2hvIGNsYWltcyB0byBiZSBwc3ljaGljKSB0byBtYWtlICJwcmVkaWN0aW9ucyIuIEluIHRoaXMgdmlkZW8sIHRoZSB3b21hbiBkZXNjcmliZXMgYSBzZXQgb2YgZXZlbnRzIHRoYXQgaGF2ZSBiZWVuIGludGVycHJldGVkIGFzIGEgZGV0YWlsZWQgZm9yZWNhc3Qgb2YKQ292aWQtMTkgcGFuZGVtaWMgdGhhdCBoYXMgaGl0IHRoZSB3b3JsZC4gSXQgaXMsIGhvd2V2ZXIsIGEgZmFrZSB2aWRlbywgYXQgbGVhc3QgYXMgcmVnYXJkcyB0aGUgZGF0ZSBvZiBpc3N1ZS4gVGhlIHZpZGVvIGhhcyBiZWVuIGJlaW5nIGRpc3NlbWluYXRlZCBvbiB0aGUKSW50ZXJuZXQgd2l0aCBhIGRhdGUgYW5kIG5vdCB0YW0KcGVyZWQgd2l0aCB0aGUgcmVhbC4%3D HTTP/1.1" 200 -


0
Fake


127.0.0.1 - - [13/May/2021 14:10:09] "GET /?algo=Regressor&granularity=Fine&text=Q2lyY3VsYXRpbmcgb24gc29jaWFsIG5ldHdvcmtzIGEgdmlkZW8gdGhhdCBzaG93cyBhbiBleGNlcnB0IGZyb20gYSBTcGFuaXNoIHRlbGV2aXNpb24gc2hvdywgc3VwcG9zZWRseSBpc3N1ZWQgRGVjZW1iZXIgMjQsIDIwMTksIGluIHdoaWNoIGl0IGFwcGVhcnMgYQp3b21hbiAod2hvIGNsYWltcyB0byBiZSBwc3ljaGljKSB0byBtYWtlICJwcmVkaWN0aW9ucyIuIEluIHRoaXMgdmlkZW8sIHRoZSB3b21hbiBkZXNjcmliZXMgYSBzZXQgb2YgZXZlbnRzIHRoYXQgaGF2ZSBiZWVuIGludGVycHJldGVkIGFzIGEgZGV0YWlsZWQgZm9yZWNhc3Qgb2YKQ292aWQtMTkgcGFuZGVtaWMgdGhhdCBoYXMgaGl0IHRoZSB3b3JsZC4gSXQgaXMsIGhvd2V2ZXIsIGEgZmFrZSB2aWRlbywgYXQgbGVhc3QgYXMgcmVnYXJkcyB0aGUgZGF0ZSBvZiBpc3N1ZS4gVGhlIHZpZGVvIGhhcyBiZWVuIGJlaW5nIGRpc3NlbWluYXRlZCBvbiB0aGUKSW50ZXJuZXQgd2l0aCBhIGRhdGUgYW5kIG5vdCB0YW0KcGVyZWQgd2l0aCB0aGUgcmVhbC4%3D HTTP/1.1" 200 -


Circulating on social networks a video that shows an excerpt from a Spanish television show, supposedly issued December 24, 2019, in which it appears a
woman (who claims to be psychic) to make "predictions". In this video, the woman describes a set of events that have been interpreted as a detailed forecast of
Covid-19 pandemic that has hit the world. It is, however, a fake video, at least as regards the date of issue. The video has been being disseminated on the
Internet with a date and not tam
pered with the real.
Regressor
0 help office dance money  ...    writing rural positive_emotion    musical
0    0      0     0     0  ...  0.0434783     0                0  0.0869565

[1 rows x 194 columns]
0
Fake


In [ ]:
predictTS20('philippin presid rodrigo dutert warn peopl flout countri lockdown slow spread coronaviru could shoot april al jazeera report polic kill man refus follow restrict queen elizabeth offer word support presid follow day facebook post claim messag hope amid coronaviru pandem queen call dutert kind leader know way filipino fortun quot continu serious look overwork may perfect truli love countri philippin worst govern actual worst citizen post flag part facebook effort combat fals news misinform news fee read partnership facebook featur fact check blogger state april blog post joe exot test posit coronaviru prison confirm ciara rourk april queen elizabeth deliv rare speech april april like facebook post say thank unit kingdom resid stay insid vow would succeed fight coronaviru pandem mention dutert philippin search quot onlin find credibl sourc show say elsewher either rate facebook post pant fire', 'Regressor')